In [1]:
import os
import shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow import keras
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

class_names = ['0', '1', '2', '3', 
               '4', '5', '6', '7', 
               '8', '9', 'A', 'B', 
               'C', 'D', 'E', 'F', 
               'G', 'H', 'K', 'L', 
               'M', 'N', 'O', 'P', 
               'R', 'S', 'T', 'U', 
               'V', 'X', 'Y', 'Z']

datagen = ImageDataGenerator(
    rescale=1.0/255, 
    validation_split=0.2,
)

train_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='training',
    class_mode='sparse'
)

validation_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='validation',
    class_mode='sparse'
)

Found 1729 images belonging to 32 classes.
Found 417 images belonging to 32 classes.


In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', 
                           padding='same',
                           input_shape=(28, 12, 3)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',
                           padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', 
                           padding='same'),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu',
                           padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='softmax')
])

2022-01-06 21:33:18.217981: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-06 21:33:18.218624: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 12, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 12, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 6, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 6, 128)        73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 6, 128)        147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 3, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2688)              0

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
class stopCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}) :
        if (logs.get('accuracy') > 0.99) :
            print('\nReached 99% accuracy so stopping training')
            self.model.stop_training = True

In [13]:
callback = stopCallback()
model.fit(train_generator, epochs=200, callbacks=[callback])

Train for 87 steps
Epoch 1/200
87/87 [==============================] - 15s 168ms/step - loss: 0.0506 - accuracy: 0.9850
Epoch 2/200
87/87 [==============================] - 14s 156ms/step - loss: 0.0458 - accuracy: 0.9855
Epoch 3/200
87/87 [==============================] - 15s 176ms/step - loss: 0.0448 - accuracy: 0.9844
Epoch 4/200
87/87 [==============================] - 15s 173ms/step - loss: 0.0362 - accuracy: 0.9861
Epoch 5/200
87/87 [==============================] - 14s 165ms/step - loss: 0.0534 - accuracy: 0.9832
Epoch 6/200
87/87 [==============================] - 15s 177ms/step - loss: 0.0316 - accuracy: 0.9873
Epoch 7/200
86/87 [============================>.] - ETA: 0s - loss: 0.0255 - accuracy: 0.9901
Reached 99% accuracy so stopping training
87/87 [==============================] - 15s 173ms/step - loss: 0.0252 - accuracy: 0.9902


In [14]:
model.evaluate(validation_generator, verbose=1)

21/21 [==============================] - 1s 58ms/step - loss: 0.1555 - accuracy: 0.9568


[0.15554114405642308, 0.95683455]

In [9]:
path1 = "../Testing_set/greenparkingchar"
path2 = "../Testing_set/greenparking2char"

all_path = [path1, path2]

final = "../charLabel/"

i = 1

for path in all_path:
    for file in os.listdir(path) :
        image_mp= mpimg.imread(path + '/' + file) / 255
        image_mp = np.expand_dims(image_mp, axis=0)
        shutil.copyfile(path + '/' + file, final + class_names[np.argmax(model.predict(image_mp))] + '/' + file)
        i += 1
        if i == 10:
            break
    if i == 10:
        break